# Overview

In [1]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from tqdm import tqdm, trange

USER_AGENT environment variable not set, consider setting it to identify your requests.


## Environment Variables

In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY']=""
os.environ['LANGCHAIN_PROJECT']="rag_demo"

## LLM

In [3]:
llm = Ollama(model="llama3.1")
print(llm.invoke("The first man on the moon was ..."))

...Neil Armstrong!

On July 20, 1969, Neil Armstrong became the first person to set foot on the Moon as part of the Apollo 11 mission. As he stepped off the lunar module Eagle and onto the Moon's surface, he famously declared: "That's one small step for man, one giant leap for mankind."


## Indexing

### Load Documents

In [4]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

### Split

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

### Embed

In [6]:
embedding_function = SentenceTransformerEmbeddings(model_name="dunzhang/stella_en_1.5B_v5") # OllamaEmbeddings(model="llama3") # dunzhang/stella_en_400M_v5 # dunzhang/stella_en_1.5B_v5
print("Download Complete.")
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_function)
retriever = vectorstore.as_retriever()

c:\Users\Tasnim\Anaconda3\envs\tasnimllm\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Tasnim\Anaconda3\envs\tasnimllm\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\Tasnim\Anaconda3\envs\tasnimllm\lib\site-packages\sentence_transformers\models\Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the defaul

Download Complete.


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


## Retrieval and Generation

### Prompt

In [7]:
prompt = hub.pull("rlm/rag-prompt")

# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful assistant that generates multiple search queries based on a single input query."),
#     ("user", "Generate multiple search queries related to: {original_query}"),
#     ("user", "OUTPUT (4 queries):")
# ])

In [8]:
print(prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [9]:
def generate_prompt(question, context):
    prompt_template=f'''You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:'''
    return prompt_template

In [10]:
print(generate_prompt("a", "b"))

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: a
Context: b
Answer:


### Post-processing

In [11]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

### Chain

In [12]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### Question

In [13]:
rag_chain.invoke("What is Task Decomposition?")

"I don't know. \n\n(Note: The provided context doesn't mention Task Decomposition, it seems to be about Maximum Inner Product Search and ANN algorithms.)"

## Similarity and Retrieved Documents

In [14]:
import tiktoken
import numpy as np

In [15]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

In [16]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [17]:
query_result = embedding_function.embed_query(question)
document_result = embedding_function.embed_query(document)

print(len(query_result))

1024


In [18]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.6351015463038355


In [19]:
# retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
# docs = retriever.get_relevant_documents("What is Task Decomposition?")
docs = retriever.get_relevant_documents("What is Task Decomposition?")
len(docs)

c:\Users\Tasnim\Anaconda3\envs\tasnimllm\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


4

In [20]:
print(docs)

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Maximum Inner Product Search (MIPS)#'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='A couple common choices of ANN algorithms for fast MIPS:'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Environment information is present in a tree structure.'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Conversatin samples:\n[\n  {\n    "role": "system",')]


# Query Translation

## Multi-Query

In [21]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads
from operator import itemgetter

### Prompt

In [22]:
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)
prompt_perspectives

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate five \ndifferent versions of the given user question to retrieve relevant documents from a vector \ndatabase. By generating multiple perspectives on the user question, your goal is to help\nthe user overcome some of the limitations of the distance-based similarity search. \nProvide these alternative questions separated by newlines. Original question: {question}'))])

In [23]:
generate_queries = (
    prompt_perspectives 
    | llm 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [24]:
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

### Retrieve

In [25]:
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})

c:\Users\Tasnim\Anaconda3\envs\tasnimllm\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


### RAG

In [26]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the following question based on this context:\n\n{context}\n\nQuestion: {question}\n'))])

In [27]:
multi_query_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

multi_query_rag_chain.invoke({"question":question})

'Task decomposition refers to the process of breaking down large tasks into smaller, manageable subgoals for efficient handling by LLM (Large Language Model) agents. This allows the agent to effectively handle complex tasks and enables efficient planning and execution of tasks.'

## RAG Fusion

### Prompt

In [28]:
rag_fusion_template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(rag_fusion_template)
prompt_rag_fusion

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are a helpful assistant that generates multiple search queries based on a single input query. \n\nGenerate multiple search queries related to: {question} \n\nOutput (4 queries):'))])

In [29]:
fusion_generate_queries = (
    prompt_rag_fusion 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [30]:
def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results


In [32]:
retrieval_chain_rag_fusion = fusion_generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

25

In [33]:
fusion_final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

print(fusion_final_rag_chain.invoke({"question":question}))

Task decomposition refers to the process of breaking down large tasks into smaller, manageable subgoals that can be efficiently handled by a Large Language Model (LLM) agent. This allows the agent to tackle complex tasks in a more organized and structured manner, leveraging its capabilities to parse and plan tasks effectively.


## Decomposition

In [34]:
decomposition_template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. Your question should only contain the generated queries. Do not provide any additional text or explanation before or after the generated queries. Your response MUST contain only the queries, numbered from 1 to 3. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(decomposition_template)
prompt_decomposition

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are a helpful assistant that generates multiple sub-questions related to an input question. \n\nThe goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. Your question should only contain the generated queries. Do not provide any additional text or explanation before or after the generated queries. Your response MUST contain only the queries, numbered from 1 to 3. \n\nGenerate multiple search queries related to: {question} \n\nOutput (3 queries):'))])

### Generate Questions

In [35]:
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})

In [36]:
for ques in questions:
    print(ques)

1. What is the role of a knowledge graph in an LLM-powered autonomous agent system?
2. How does a natural language understanding component contribute to the functionality of an LLM-powered autonomous agent system?
3. What are the key characteristics of a decision-making module in an LLM-powered autonomous agent system?


In [37]:
decomposition_qa_pair_template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt_qa_pair = ChatPromptTemplate.from_template(decomposition_qa_pair_template)

decomposition_prompt_qa_pair

ChatPromptTemplate(input_variables=['context', 'q_a_pairs', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'q_a_pairs', 'question'], template='Here is the question you need to answer:\n\n\n --- \n {question} \n --- \n\n\nHere is any available background question + answer pairs:\n\n\n --- \n {q_a_pairs} \n --- \n\n\nHere is additional context relevant to the question: \n\n\n --- \n {context} \n --- \n\n\nUse the above context and any background question + answer pairs to answer the question: \n {question}\n'))])

In [38]:
def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

In [39]:
q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt_qa_pair
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs}) 
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

print(answer)

Based on the provided context and the relevant background information, I will attempt to answer the question.

The key characteristics of a decision-making module in an LLM-powered autonomous agent system can be inferred by looking at the role of tool use, natural language understanding (NLU), and knowledge graph integration in such systems.

From the context provided, it appears that the decision-making module plays a crucial role in enabling the LLM-powered autonomous agent system to make informed decisions based on user input, external information gathered through tool calls, and the system's internal model weights.

The mention of Boiko et al. (2023) suggests that the agent uses tools "to browse the Internet, read documentation, execute code, call robotics experimentation APIs" and more. This implies that the decision-making module is responsible for deciding which external tools or APIs to utilize based on user input and the system's internal understanding.

The answer can also be

In [40]:
def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

In [41]:
prompt_rag = hub.pull("rlm/rag-prompt")
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [42]:
def format_qa_pairs_individual(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs_individual(questions, answers)

# Prompt
template_individual = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

individual_final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

print(individual_final_rag_chain.invoke({"context":context,"question":question}))

Based on the provided context, I would say that the main components of an LLM-powered autonomous agent system include:

1. **Large Language Model (LLM)**: The core component that provides the ability to understand and generate human-like text.
2. **External API access**: Allows the agent to gather extra information from external sources.
3. **Current data access**: Enables the agent to leverage current data for informed decision-making.
4. **Code execution capabilities**: Empowers the agent to execute code and perform tasks that may not be possible with just the LLM's abilities.
5. **Proprietary source utilization**: Allows the agent to tap into proprietary sources of information, if available.

These components work together to enable autonomous agents to make informed decisions and take actions accordingly.


## Step Back

In [43]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

In [44]:
examples = [
    {
        "input": "A hotel employs cleaners and receptionists. Cleaners earn $500 per week and receptionists earn $350 per week. The hotel requires a minimum of 100 workers of whom at least 20 must be receptionists. To keep the hotel clean and running smoothly, the number of receptionists should be at least a third of the number of cleaners. The hotel wants to keep the weekly wage bill below $30000. Formulate a LP to minimize the wage bill.",
        "output": "How can linear programming be applied to determine the optimal number of employees in different roles to minimize costs under given constraints?",
    },
    {
        "input": "You are playing a game where a short shot is worth 2 points and a long shot is worth 5 points. In total, you can take at most 14 shots. You must take at least 5 short shots and 2 long shots, but time restricts taking more than 8 of either type. How many of each shot must you take, assuming all your shots get points, to maximize your score? What is your maximum score?",
        "output": "How can linear programming be used to maximize the total score in a game with constraints on the number and type of shots taken?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "Generate Step-back question for this Question: {question}. You MUST provide only the step-back question. Do not provide any additional text before or after the question. Be concise."),
    ]
)

print(prompt)

input_variables=['question'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:')), FewShotChatMessagePromptTemplate(examples=[{'input': 'Could the members of The Police perform lawful arrests?', 'output': 'what can the members of The Police do?'}, {'input': 'Jan Sindel’s was born in what country?', 'output': 'what is Jan Sindel’s personal history?'}], example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], template='{output}'))])), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Generate Step-back question for this Question: {question}.

In [45]:
generate_queries_step_back = prompt | llm | StrOutputParser()
question = "What is task decomposition for LLM agents?"

print(generate_queries_step_back.invoke({"question": question}))

What are the fundamental concepts involved in training large language models?


In [46]:
# Response prompt 
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

step_back_chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
)

print(step_back_chain.invoke({"question": question}))

Task decomposition refers to the process of breaking down complex tasks into smaller, more manageable sub-tasks that can be executed by Large Language Model (LLM) agents. This approach involves identifying the key components or steps required to complete a specific task and then creating separate prompts or inputs for each sub-task.

By decomposing tasks in this way, LLM agents can focus on one aspect of the task at a time, which can improve their performance, efficiency, and overall quality of output. Task decomposition is particularly useful when working with complex or open-ended tasks that require multiple steps or interactions to complete.

In the context of LLMs, task decomposition can involve several key components:

1. **Sub-task identification**: Identifying the individual sub-tasks required to complete a larger task.
2. **Prompt engineering**: Crafting specific prompts or inputs for each sub-task that can be executed by the LLM agent.
3. **Model selection**: Choosing the most

## HyDE

### HyDE document generation

In [47]:
hyde_template = """Please write a scientific paper passage to answer the question. You output should contain only the scientific paper passage. DO NOT provide any additional text before or after it.
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(hyde_template)

print(prompt_hyde)

input_variables=['question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Please write a scientific paper passage to answer the question. You output should contain only the scientific paper passage. DO NOT provide any additional text before or after it.\nQuestion: {question}\nPassage:'))]


In [48]:
generate_docs_for_retrieval = (
    prompt_hyde | llm | StrOutputParser() 
)

# Run
question = "What is task decomposition for LLM agents?"
print(generate_docs_for_retrieval.invoke({"question":question}))

Task decomposition is a crucial concept in the development and evaluation of Large Language Model (LLM) agents, particularly those designed for complex tasks such as conversational dialogue systems, question-answering, and text summarization. In essence, task decomposition refers to the process of breaking down a given task into smaller, manageable sub-tasks that can be individually addressed by the LLM agent.

The primary motivation behind task decomposition in LLM agents is to facilitate more efficient learning and processing of complex tasks. By decomposing a task into smaller components, developers can design more targeted training data and evaluation protocols, which can lead to improved model performance and reduced computational requirements. Moreover, task decomposition enables researchers to identify and address specific knowledge gaps or biases within the model, thereby enhancing its overall reliability and generalizability.

Task decomposition in LLM agents typically involve

### Retrieve

In [49]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='ReAct (Yao et al. 2023) integrates reasoning and acting within LLM by extending the action space to be a combination of task-specific discrete actions and the language space. The former enables LLM to interact with the environment (e.g. use Wikipedia search API), while the latter prompting LLM to'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='The system comprises of 4 stages:\n(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks. There are four attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot examples to guide LLM to do task parsing and planning.'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, 

In [50]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

hyde_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

hyde_rag_chain.invoke({"context":retireved_docs,"question":question})

'According to the provided context, task decomposition refers to the process of breaking down large tasks into smaller, manageable subgoals by an LLM (Large Language Model) agent. This allows for efficient handling of complex tasks.'

# Routing

## Logical Routing

In [51]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Literal
from langchain_core.prompts import PromptTemplate

### Data Model

In [54]:
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
    ...,
    description="Given a user question choose which datasource would be most relevant for answering their question",
)


### LLM for Structured Output

In [55]:
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source.
DataSources are: [python_docs, js_docs, golang_docs]
Your response should contain only the data source name. DO NOT provide any additional text before or after the data source name."""

structured_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = structured_prompt | llm

In [56]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

print(result)

python_docs


In [57]:
def choose_route(result):
    if "python_docs" in result.lower():
        ### Logic here 
        return "chain for python_docs"
    elif "js_docs" in result.lower():
        ### Logic here 
        return "chain for js_docs"
    else:
        ### Logic here 
        return "golang_docs"

router_chain = router | RunnableLambda(choose_route)

In [58]:
print(router_chain.invoke({"question": question}))

chain for python_docs


## Semantic Routing

### Two prompts

In [59]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

### Embed Prompts

In [60]:
prompt_templates = [physics_template, math_template]
prompt_embeddings = embedding_function.embed_documents(prompt_templates)

### Route Question to Prompt

In [61]:
def prompt_router(input):
    # Embed question
    query_embedding = embedding_function.embed_query(input["query"])
    # Compute similarity
    s_1 = cosine_similarity(query_embedding, embedding_function.embed_query(math_template))
    s_2 = cosine_similarity(query_embedding, embedding_function.embed_query(physics_template))
    if(s_1>=s_2):
        most_similar=math_template
    else:
        most_similar=physics_template
    print(most_similar)
    return PromptTemplate.from_template(most_similar)

In [62]:
semantic_chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)

In [63]:
print(semantic_chain.invoke("What is the equation of root mean squared error?"))

You are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into their component parts, answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}
To find the equation of Root Mean Squared Error (RMSE), we can start by breaking it down into its component parts:

1. **Mean**: The mean is the average value of a set of numbers. It's calculated by summing up all the values and dividing by the total count.

Mathematically, if we have a set of N values, {x1, x2, ..., xN}, then the mean (μ) is:

μ = (x1 + x2 + ... + xN) / N

2. **Variance**: The variance measures how spread out the numbers are from their mean value.

For our set of values {x1, x2, ..., xN}, we calculate the variance (σ²) as:

σ² = [(x1 - μ)² + (x2 - μ)² + ... + (xN - μ)²] / N

3. **Root**: Taking the square root of a value is denoted by √.

4. **Mean Squared Error** (MSE): The Mean Square